In [13]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_path = os.path.abspath("..")

sys.path.append(project_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from itertools import combinations

from bs4 import BeautifulSoup as bs

from src.scrap_hrefs import get_units_hrefs
from src.utils import fetch_page_content

In [3]:
aoe_wiki_url = "https://ageofempires.fandom.com/"
units_url = aoe_wiki_url + "wiki/Unit_(Age_of_Empires_III)"
unit_list_html = fetch_page_content(units_url)
href_soup = bs(unit_list_html, "html.parser")
main_page = href_soup.find('main', class_='page__main')


content = href_soup.find('div', class_='mw-parser-output')
h2s = content.find_all('h2', recursive=False)
sections = [h2.find('span', class_='mw-headline', recursive=False).text for h2 in h2s]
section_unit_hrefs = {section: get_units_hrefs(section, content) for section in sections}

In [4]:
# Units in more than one building
unique_units = {}
for key, values in section_unit_hrefs.items():
    unique_units[key] = set(values.keys())

# Generar todas las posibles parejas de llaves
key_pairs = combinations(unique_units.keys(), 2)

for k1, k2 in key_pairs:
    intersection = unique_units[k1] & unique_units[k2]
    if len(intersection) > 0:
        pair = (k1, k2)
        print(pair, " :", intersection)

('Explorer/War Chief/Monk/General', 'Town Center')  : {'Envoy'}
('Town Center', 'Trading Post/Native Embassy/Tambo')  : {'Chasqui'}
('Livestock Pen/Farm', 'Village')  : {'Goat'}
('Dock/Port', 'Revolutionary Variants')  : {'Fire Ship'}
('Dock/Port', 'Untrainable')  : {'Fire Ship'}
('Trading Post/Native Embassy/Tambo', 'Community Plaza')  : {'Healer'}
('Church/Mosque/Meeting House/Cathedral', 'Basilica')  : {'Priest', 'Spy'}
('Artillery Foundry', 'Factory')  : {'Rocket'}
('Kallanka', 'Community Plaza')  : {'Maceman'}
('Revolutionary Variants', 'Untrainable')  : {'Boneguard Swordsman', "Lizzie's Flagship", 'Greta', 'Great Plains Chief', 'Horse', 'Treasure Ship', 'Native American Chief', 'Buccaneer Captain', 'Aztec Chief', 'White Wolf', 'Railroad Worker', 'Flat-bottomed Boat', 'White Buffalo', 'Cherokee Horse Archer', 'Dinghy', 'Black Powder Wagon', 'Boneguard Musketeer', 'Great Cannon', 'Fire Ship', 'White Jaguar', 'Outlaw Musketeer', "Morgan's Flagship", 'Cherokee Archer', 'Miner', 'Fier